# Handwritten Digit Recognition with MNIST

In this project, we are going to build a handwritten digit recognition neural network from scratch, using just python (numpy) and maths. We will be using the downloaded version of the MNIST dataset from Kaggle, and here's how it works. 

### Preparation on the dataset

In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import struct

We have the IDX file format here (for the MNIST dataset). Hence, we have to convert it to a CSV file first, which can make things easier when it comes to setting up arrays.

There are 4 files under the archive zip folder after you extract it. For training, we can use `train-images.idx3-ubyte`, which are the images themselves, merging with `train-labels.idx1-ubyte`, which are the labels of the images in the dataset in order. 
(In the original downloaded dataset, the file for images and the labels are separated. Hence, we have to merge them together.) 

In [11]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

images = read_idx('C:/Users/zoe/OneDrive/Deep Learning Projects/Handwritten Digits/archive/train-images.idx3-ubyte')

images_flat = images.reshape(images.shape[0], -1)
df1 = pd.DataFrame(images_flat)

df1.to_csv('train-images.csv', index=False)

In [12]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

labels = read_idx('C:/Users/zoe/OneDrive/Deep Learning Projects/Handwritten Digits/archive/train-labels.idx1-ubyte')

df2 = pd.DataFrame(labels, columns=['label'])

df2.to_csv('train-labels.csv', index=False)

In [13]:
# Combining the 2 datasets
df1 = pd.read_csv('C:/Users/zoe/OneDrive/Deep Learning Projects/Handwritten Digits/train-images.csv')
df2 = pd.read_csv('C:/Users/zoe/OneDrive/Deep Learning Projects/Handwritten Digits/train-labels.csv')

data = pd.concat([df2, df1], axis=1)

data.to_csv('combined-dataset.csv', index=False)

data.head()

,label,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We now have to make this into an array. We see that there are 784 inputs for each row (excluding the label column), each representing a pixel in that 28*28 sized photo of handwritten digit. 

We will try to shuffle these arrays for making the whole datastructure more "random" for the training (you can think of it as less easy for the neural network to recognise those patterns).

In [50]:
nodes = np.array(data)
np.random.shuffle(nodes)

nodes.shape
nodes

array([[3, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], shape=(60000, 785))

Now we have a bunch of arrays that represents the **input layer**. Using `nodes.shape` we can check the dimensions of these arrays again (60000 is the number of rows, 785 is the number of columns).

### Constructing the neural network

Remark: we need to split nodes dataset into 3 types of datasets: 1. training dataset; 2. validation dataset; 3. testing dataset. For validation dataset, it is to check that the neural network didn't memorise the training data and it has really done some learning from it. 

We start by picking out 2000 validation dataset. For the equation in neural networks (more explanation in medium blog post/ readme file), it should be something like y = activation function on the summation of (weight matrix * nodes + bias). 

A very important remark is that if we don't transpose the node sample (reason would be explained in blog/ readme file as well), we would not get the Y_valid and X_valid we wanted. 

In [51]:
nodes_valid = nodes[:2000].T

# Transpose th= nodes_validle as it is originally a row vector -> we want it to be a column vecto= nodes_valid = nodes_valid.T

Y_valid = nodes_valid[0]
X_valid = nodes_valid[1:]

# Check the contents in X_valid and Y_valid
print(Y_valid)
print(X_valid)

# Check the size of X_valid and Y_valid
# For X_valid, we just need the first row vector to check the size
print(Y_valid.size)
print(X_valid[:,0].size)

[3 6 1 ... 0 2 1]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
2000
784


Here, Y_valid are the final results we get from the **output layer**, and X_valid is somethng we called an **activation**, which is a value in those input layer nodes that we have inputed in it. 

Now, we prepare the training dataset. As the testing dataset is included in another file under the archive foler: `t10k-images.idx3-ubyte` and `t10k-labels.idx1-ubyte`, so we just set the remaining datasets here to be the training dataset. For the ratio between the number (size) of these datasets, it's better to follow 2:6:2 (valid: train: test).

In [52]:
nodes_train = nodes[2001: 8001].T

Y_train = nodes_train[0]
X_train = nodes_train[1:]

print(Y_train)
print(X_train)

print(Y_train.size)
print(X_train[:,0].size)

[5 4 1 ... 7 5 6]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
6000
784


In [ ]:
def init_parameters():
    W1 = np.random.rand(10, 784) - 0.5 
    b1 = np.random.rand(10, 1) - 0.5 
    W2 = np.random.rand(10, 784) - 0.5 
    b2 = np.random.rand(10, 1) - 0.5 
    return W1, b1, W2, b2

def relu(z):
    return np.maximum(0, z)

def softmax(z):
    exp_z = np.exp(z - np.max(z))  # Subtract max(z) for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):
    # Doing matrix operations using dot product
    Z1 = W1.dot(X) + b1
    A1 = relu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    dZ2 = A2 - Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * (Z1 > 0)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def gradient_descent (X, Y, iterations, alpha): 
    W1, b1, W2, b2 = init_parameters()
    for i in range(iterations): 
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print('Iteration:', i)
            print("Accuracy:", np.mean(np.argmax(A2, axis=0) == Y))
    return W1, b1, W2, b2